### Import the necessary dependencies

In [1]:
from glob import glob 
import nibabel as nib
import numpy as np
import dicom2nifti
import random
import shutil
import os

### Script to create different directories each with the same number of DICOM images or slices

### Adjust the "slices_per_folder" variable as per preference

In [16]:
def format_slices_per_folder(slices_per_folder, input_path, output_path):
    for patient_path in glob(input_path + "/*"):
        patient_num = os.path.basename(os.path.normpath(patient_path))

        folders_num = int(len(glob(patient_path + "/*")) / slices_per_folder)

        for folder_num in range(folders_num):
            output_path_name = os.path.join(output_path, patient_num + "_" + str(folder_num))

            os.mkdir(output_path_name)

            for i, file in enumerate(glob(patient_path + "/*")):
                if i == slices_per_folder:
                    break

                shutil.move(file, output_path_name)

format_slices_per_folder(slices_per_folder = 50, input_path = "G:/DL for Liver Segmentation/renewed_dicom_file/labels", output_path = "G:/DL for Liver Segmentation/renewed_dicom_group/labels")

### Script to ensure that all directories contain the same number of slices

In [36]:
def count_elements_in_folders(directory_path):
    folder_counts = {}
    
    for folder in glob(os.path.join(directory_path, '*')):
        if os.path.isdir(folder):
            folder_name = os.path.basename(folder)
            element_count = len(glob(os.path.join(folder, '*')))
            folder_counts[folder_name] = element_count
            
    return folder_counts

def sort_numerically(folders):
    # Extract numbers from folder names for sorting
    return sorted(folders, key=lambda x: int(''.join(filter(str.isdigit, x)) or 0))

# Example usage of the new feature:
directory_path = "G:/DL for Liver Segmentation/dicom_groups/labels"
folder_counts = count_elements_in_folders(directory_path)

sorted_folders = sort_numerically(folder_counts.keys())

for folder in sorted_folders:
    if folder_counts[folder] != 50:
        print(f"Folder '{folder}' does not contain 50 elements.")

### Script to convert the DICOM series of patient images in each folder into compressed .nii files

In [26]:
def dicom_to_nifti_conversion_for_images(images_input_path, images_output_path):

    list_of_images = glob(images_input_path)
    
    for patient_index in list_of_images:
        patient_num = os.path.basename(os.path.normpath(patient_index))
        dicom2nifti.dicom_series_to_nifti(patient_index, os.path.join(images_output_path, patient_num + ".nii.gz"))

dicom_to_nifti_conversion_for_images(images_input_path = "G:/DL for Liver Segmentation/dicom_groups/images/*", images_output_path = "G:/DL for Liver Segmentation/final_nifti_files/images")

### Script to convert the DICOM series of patient segmentations in each folder into compressed .nii files

In [ ]:
def dicom_to_nifti_conversion_for_labels(labels_input_path, labels_output_path):

    list_of_labels = glob(labels_input_path)
    
    for patient_index in list_of_labels:
        patient_num = os.path.basename(os.path.normpath(patient_index))
        output_file = dicom2nifti.dicom_series_to_nifti(patient_index, os.path.join(labels_output_path, patient_num + ".nii.gz"))

        # Added a try and except method because label conversion from DICOM series into .nii.gz files typically involves slice increment errors 
        try:
            dicom2nifti.dicom_series_to_nifti(patient_index, output_file)
        except Exception as e:
            print(f"Error processing {patient_index}: {e}")

dicom_to_nifti_conversion_for_labels(labels_input_path = "G:/DL for Liver Segmentation/dicom_groups/labels/*", labels_output_path = "G:/DL for Liver Segmentation/final_nifti_files/labels")

In [ ]:
import pydicom

def check_dicom_metadata(directory):
    dicom_files = sorted(glob(os.path.join(directory, "*.dcm")))
    
    for dicom_file in dicom_files:
        ds = pydicom.dcmread(dicom_file)
        print(f"File: {dicom_file}")
        print(f"  Instance Number: {ds.InstanceNumber}")
        print(f"  Slice Location: {ds.SliceLocation if 'SliceLocation' in ds else 'N/A'}")
        print(f"  Image Position (Patient): {ds.ImagePositionPatient if 'ImagePositionPatient' in ds else 'N/A'}")
        print(f"  Slice Thickness: {ds.SliceThickness if 'SliceThickness' in ds else 'N/A'}")
        print(f"  Spacing Between Slices: {ds.SpacingBetweenSlices if 'SpacingBetweenSlices' in ds else 'N/A'}")
        print()

# Replace with the specific directory of the problematic series
check_dicom_metadata("G:/DL for Liver Segmentation/dicom_groups/labels/liver_102_3")

### Script to check which image files don't contain any labels (segmentations) of the liver - such files can potentially be moved to a separate directory as they may not be beneficial for the final deep learning model

In [ ]:
def find_empty_slices(input_labels_path):
    files_list = glob(input_labels_path)

    file_count_with_liver_visible = 0
    file_count_without_liver = 0

    files_with_liver = []
    files_without_liver = []

    for file in files_list:

        nifti_file = nib.load(file)
        frame_data = nifti_file.get_fdata()

        num_of_items = np.unique(frame_data)

        if len(num_of_items) > 1:
            files_with_liver.append(file)

            file_count_with_liver_visible += 1
        
        else:
            files_without_liver.append(file)

            file_count_without_liver += 1

            # shutil.move(file, output_labels_path)

    print(f"Total # of files showcasing liver: {file_count_with_liver_visible}")
    print(f"Total # of files not showcasing liver: {file_count_without_liver}")

find_empty_slices(input_labels_path = "G:/DL for Liver Segmentation/final_nifti_files/labelsVal/*")

In [ ]:
def create_val_dataset(input_labels_path, output_labels_path, liver_samples = 90, non_liver_samples = 20):
    # Ensure the output directory exists
    os.makedirs(output_labels_path, exist_ok=True)

    files_list = glob(input_labels_path)

    file_count_with_liver_visible = 0
    file_count_without_liver = 0

    files_with_liver = []
    files_without_liver = []

    for file in files_list:
        nifti_file = nib.load(file)
        frame_data = nifti_file.get_fdata()

        num_of_items = np.unique(frame_data)

        if len(num_of_items) > 1:
            files_with_liver.append(file)
            file_count_with_liver_visible += 1
        else:
            files_without_liver.append(file)
            file_count_without_liver += 1

    print(f"Total # of files showcasing liver: {file_count_with_liver_visible}")
    print(f"Total # of files not showcasing liver: {file_count_without_liver}")

    # Randomly select files
    selected_files_with_liver = random.sample(files_with_liver, min(liver_samples, len(files_with_liver)))
    selected_files_without_liver = random.sample(files_without_liver, min(non_liver_samples, len(files_without_liver)))

    # Combine the selected files
    selected_files = selected_files_with_liver + selected_files_without_liver

    # Move the selected files to the output directory
    for file in selected_files:
        shutil.move(file, os.path.join(output_labels_path, os.path.basename(file)))

    print(f"Moved {len(selected_files)} files to {output_labels_path}")

# Example usage
create_val_dataset(
    input_labels_path = "G:/DL for Liver Segmentation/final_nifti_files/labels/*",
    output_labels_path = "G:/DL for Liver Segmentation/final_nifti_files/labelsVal",
    liver_samples = 90,
    non_liver_samples = 20
)


In [ ]:
def move_matching_files(first_input_dir, second_input_dir, output_dir):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Get the list of files in the first input directory
    first_dir_files = set(os.listdir(first_input_dir))

    # Iterate through the files in the second input directory
    for file_name in os.listdir(second_input_dir):
        if file_name in first_dir_files:
            # Construct full file paths
            source_file = os.path.join(second_input_dir, file_name)
            destination_file = os.path.join(output_dir, file_name)
            
            # Move the file
            shutil.move(source_file, destination_file)

            print(f"Moved {file_name} to {output_dir}")

# Example usage
move_matching_files(
    first_input_dir="G:/DL for Liver Segmentation/final_nifti_files/labelsVal",
    second_input_dir="G:/DL for Liver Segmentation/final_nifti_files/images",
    output_dir="G:/DL for Liver Segmentation/final_nifti_files/imagesVal"
)